In [1]:
# Day 1 — Ingestion (Alpha Vantage)

# Goal: Download **daily adjusted** OHLCV for several tickers (US + TSX) and save:
# - Raw response as JSON → `data/raw/`
# - Clean CSV (date, open, high, low, close, adj_close, volume) → `data/interim/`

In [2]:
import os
from dotenv import load_dotenv
from pathlib import Path

load_dotenv(Path.cwd().parents[0] / ".env")
API_KEY = os.getenv("ALPHAVANTAGE_KEY")
print("Loaded:", bool(API_KEY), "| startswith:", (API_KEY[:4] + "****") if API_KEY else None)
assert API_KEY and API_KEY != "REPLACE_ME", "Set a real ALPHAVANTAGE_KEY in .env and re-run."

Loaded: True | startswith: VZEO****


In [3]:
import requests
import pandas as pd

def symbol_search(keywords: str, max_rows: int = 10) -> pd.DataFrame:
    url = "https://www.alphavantage.co/query"
    params = {"function": "SYMBOL_SEARCH", "keywords": keywords, "apikey": API_KEY}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    data = r.json().get("bestMatches", [])
    df = pd.DataFrame(data)
    if df.empty:
        return df
    df.columns = [c.split(". ", 1)[-1] for c in df.columns]
    return df.head(max_rows)

# Example
symbol_search("SPY")


,symbol,name,type,region,marketOpen,marketClose,timezone,currency,matchScore
0,SPY,SPDR S&P 500 ETF Trust,ETF,United States,09:30,16:00,UTC-04,USD,1.0000
1,SPY1.FRK,SSgA SPDR ETFs Europe I Public Limited Company...,ETF,Frankfurt,08:00,20:00,UTC+02,EUR,0.6667
2,SPY2.FRK,SSgA SPDR ETFs Europe I Public Limited Company...,ETF,Frankfurt,08:00,20:00,UTC+02,EUR,0.6667
3,SPY3.LON,Leverage Shares 3x Long US 500 ETP Securities,ETF,United Kingdom,08:00,16:30,UTC+01,USD,0.6667
4,SPY4.FRK,SSgA SPDR S&P 400 US Mid Cap,ETF,Frankfurt,08:00,20:00,UTC+02,EUR,0.6667
5,SPY4.LON,SPDR® S&P 400 US Mid Cap UCITS ETF,ETF,United Kingdom,08:00,16:30,UTC+01,USD,0.6667
6,SPY1.DEX,SPDR® S&P 500 Low Volatility UCITS ETF,ETF,XETRA,08:00,20:00,UTC+02,EUR,0.6000
7,SPY2.DEX,SPDR® Dow Jones Global Real Estate UCITS ETF U...,ETF,XETRA,08:00,20:00,UTC+02,EUR,0.6000
8,SPY4.DEX,SSgA SPDR S&P 400 US Mid Cap,ETF,XETRA,08:00,20:00,UTC+02,EUR,0.6000
9,SPY4.PAR,SSgA SPDR S&P 400 US Mid Cap,ETF,Paris,09:00,17:30,UTC+02,EUR,0.6000


In [4]:
import json, time, requests, pandas as pd
from pathlib import Path

BASE = Path.cwd().parents[0]
DATA_RAW = BASE / "data" / "raw"
DATA_INTERIM = BASE / "data" / "interim"
DATA_RAW.mkdir(parents=True, exist_ok=True)
DATA_INTERIM.mkdir(parents=True, exist_ok=True)

def fetch_daily_free(symbol: str,
                     outputsize: str = "full",
                     min_sleep: float = 65.0,    # safe for 5/min
                     max_retries: int = 5):
    """
    Use Alpha Vantage TIME_SERIES_DAILY (free).
    We map adj_close = close for now (no paid adjustments).
    """
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_DAILY",   # <-- free endpoint
        "symbol": symbol,
        "apikey": API_KEY,
        "outputsize": outputsize,
    }
    for attempt in range(1, max_retries+1):
        r = requests.get(url, params=params, timeout=30)
        r.raise_for_status()
        data = r.json()
        # Save raw for debugging
        (DATA_RAW / f"{symbol}.json").write_text(json.dumps(data, indent=2)[:1_000_000])

        # Happy path
        key = next((k for k in data if "Time Series" in k), None)
        if key:
            ts = data[key]
            rows = []
            for dt, row in sorted(ts.items()):
                rows.append({
                    "date": dt,
                    "open": row.get("1. open"),
                    "high": row.get("2. high"),
                    "low":  row.get("3. low"),
                    "close":row.get("4. close"),
                    "adj_close": row.get("4. close"),   # no adjusted on free daily API, so we will use close to fill the column
                    "volume": row.get("5. volume"),
                })
            df = pd.DataFrame(rows)
            df.to_csv(DATA_INTERIM / f"{symbol}.csv", index=False)
            print(f"Saved {symbol}: {len(df)} rows.")
            time.sleep(min_sleep)  # be polite before next call
            return

        # Throttle / premium / error -> backoff and retry
        msg = data.get("Note") or data.get("Information") or data.get("Error Message")
        print(f"[Attempt {attempt}] {symbol}: {msg}")
        time.sleep(min_sleep + attempt*10)

    raise RuntimeError(f"Failed to fetch {symbol} after {max_retries} retries.")


In [5]:
def save_from_stooq(symbol: str, stooq_symbol: str):
    """
    symbol: how you want the CSV named locally (e.g., 'SPY')
    stooq_symbol: Stooq's code, e.g., 'spy.us' for SPY
    """
    url = f"https://stooq.com/q/d/l/?s={stooq_symbol}&i=d"
    df = pd.read_csv(url)  # Date, Open, High, Low, Close, Volume
    df.columns = [c.lower() for c in df.columns]
    df["adj_close"] = df["close"]       # Stooq has no separate adjusted
    df = df[["date","open","high","low","close","adj_close","volume"]]
    df.to_csv(DATA_INTERIM / f"{symbol}.csv", index=False)
    print(f"Saved {symbol} from Stooq ({stooq_symbol}).")

In [6]:
#Testing save_from_stooq
save_from_stooq("SPY", "spy.us")

Saved SPY from Stooq (spy.us).


In [7]:
#Running the ingestion for one symbol
try:
    fetch_daily_free("SPY", outputsize="compact", min_sleep=65.0)
except Exception as e:
    print("AV SPY failed → trying Stooq:", e)
    save_from_stooq("SPY", "spy.us")

Saved SPY: 100 rows.


In [ ]:
#Testing with multiple symbols
for sym in ["QQQ", "VFV.TO", "XEQT.TO"]:
    try:
        fetch_daily_free(sym, outputsize="full", min_sleep=65.0)
    except Exception as e:
        print(f"AV {sym} failed → trying Stooq:", e)
        # For TSX tickers Stooq often uses different codes; you can skip fallback or map if available.

Saved QQQ: 6506 rows.
